# GPU Programming with Python - PyCUDA

In [1]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import numpy as np

## Test it out

In [2]:
print(f"{cuda.Device.count()} device(s) found")
for i in range(cuda.Device.count()):
    dev = cuda.Device(i)
    print(f"Device {i}: {dev.name()}")
    a, b = dev.compute_capability()
    print(f"  Compute capability: {a}.{b}")
    print(f"  Total memory: {dev.total_memory() / 1024} KB")

1 device(s) found
Device 0: Quadro P4000
  Compute capability: 6.1
  Total memory: 8308736.0 KB


## Matrix * 2
1. set up your data (array/vector, matrix) on the host, setting type to `np.float32`
1. allocate space on the GPU's memory and copy the data to it (to device)
1. write the key computational kernel for the GPU
1. get the function and call it, give as parameters the pointer to your data on the GPU and the block size
1. create a new variable to contain the data from the GPU and copy it (to host)

In [3]:
"""Create a random matrix of 5x5 and convert to float32 for GPU architecture"""
a = np.random.randn(5,5).astype(np.float32)
a

array([[-1.2237811 ,  1.7522274 ,  0.2198983 ,  0.6134487 , -0.8579877 ],
       [ 0.22636674, -1.1725038 , -0.35058454, -0.97915286,  1.4445748 ],
       [-0.41134974,  0.96850103,  1.0174406 , -0.41382772, -0.6064575 ],
       [ 0.43275827,  0.06282279,  0.0935237 , -0.9783392 , -1.5713155 ],
       [ 0.18036895, -1.4524317 ,  1.1295315 ,  0.38706082,  1.3106147 ]],
      dtype=float32)

In [4]:
"""Allocate space on the GPU's memory and copy the data to it"""
a_gpu = cuda.mem_alloc(a.nbytes)
cuda.memcpy_htod(a_gpu, a)

In [5]:
""""Write the key computational kernel for the GPU, double the matrix given"""
mod = SourceModule("""
    __global__ void doubleMatrix(float *a) {
        int idx = threadIdx.x + threadIdx.y * blockDim.x;
        a[idx] *= 2;
    }
""")

In [6]:
func = mod.get_function("doubleMatrix")
func(a_gpu, block=(5,5,1))

In [7]:
a_doubled = np.empty_like(a)
cuda.memcpy_dtoh(a_doubled, a_gpu)

In [8]:
print("------------- ORIGINAL MATRIX -------------")
print(a)
print("---- DOUBLED MATRIX AFTER COMPUTATION -----")
print(a_doubled)

------------- ORIGINAL MATRIX -------------
[[-1.2237811   1.7522274   0.2198983   0.6134487  -0.8579877 ]
 [ 0.22636674 -1.1725038  -0.35058454 -0.97915286  1.4445748 ]
 [-0.41134974  0.96850103  1.0174406  -0.41382772 -0.6064575 ]
 [ 0.43275827  0.06282279  0.0935237  -0.9783392  -1.5713155 ]
 [ 0.18036895 -1.4524317   1.1295315   0.38706082  1.3106147 ]]
---- DOUBLED MATRIX AFTER COMPUTATION -----
[[-2.4475622   3.5044549   0.4397966   1.2268974  -1.7159754 ]
 [ 0.4527335  -2.3450077  -0.7011691  -1.9583057   2.8891497 ]
 [-0.8226995   1.9370021   2.034881   -0.82765543 -1.212915  ]
 [ 0.86551654  0.12564558  0.1870474  -1.9566784  -3.142631  ]
 [ 0.3607379  -2.9048634   2.259063    0.77412164  2.6212294 ]]


## An even easier introduction to CUDA
https://devblogs.nvidia.com/even-easier-introduction-cuda/  
Here's the code in C++, we'll need to port that to Python:

```c++
#include <iostream>
#include <math.h>

// function to add the elements of two arrays
void add(int n, float *x, float *y)
{
  for (int i = 0; i < n; i++)
      y[i] = x[i] + y[i];
}

int main(void)
{
  int N = 1<<20; // 1M elements

  float *x = new float[N];
  float *y = new float[N];

  // initialize x and y arrays on the host
  for (int i = 0; i < N; i++) {
    x[i] = 1.0f;
    y[i] = 2.0f;
  }

  // Run kernel on 1M elements on the CPU
  add(N, x, y);

  // Check for errors (all values should be 3.0f)
  float maxError = 0.0f;
  for (int i = 0; i < N; i++)
    maxError = fmax(maxError, fabs(y[i]-3.0f));
  std::cout << "Max error: " << maxError << std::endl;

  // Free memory
  delete [] x;
  delete [] y;

  return 0;
}
```

### Here's that same code in Python

In [9]:
import time

In [ ]:
def add(n, x, y):
    """Function to add the elements of two arrays"""
    for i in range(0, n):
        y[i] = x[i] + y[i]

In [ ]:
N = 1<<20 # 1M elements
x = np.ones(N, dtype=np.float32)
y = np.full(N, 2)

In [ ]:
start = time.time()
add(N, x, y)
print(f"---- {time.time() - start} seconds ----")

In [ ]:
start = time.time()
max_error = 0.0
for i in range(0, N):
    global max_error
    max_error = max(max_error, (y[i] - 3.0))
print(f"Max error: {max_error}")
print(f"---- {time.time() - start} seconds ----")

x = []
y = []

### Now let's get this running on the GPU
1. set up your data (array/vector, matrix) on the host, setting type to `np.float32`
1. allocate space on the GPU's memory and copy the data to it (to device)
1. write the key computational kernel for the GPU
1. get the function and call it, give as parameters the pointer to your data on the GPU and the block size
1. synchronize with the device: wait for GPU to finish before accessing on host
1. create a new variable to contain the data from the GPU and copy it (to host)
1. free up the memory on the device

In [10]:
N = 1<<20 # 1M elements
x = np.ones(N, dtype=np.float32)
y = np.full(N, 2)

In [11]:
x_gpu = cuda.mem_alloc(x.nbytes)
y_gpu = cuda.mem_alloc(y.nbytes)

In [12]:
cuda.memcpy_htod(x_gpu, x)
cuda.memcpy_htod(y_gpu, y)

In [14]:
mod = SourceModule("""
    __global__ void add(float *x, float *y)
    {
        int n = 1<<20;
        for (int i = 0; i < n; i++)
            y[i] = x[i] + y[i];
    }
""")

In [22]:
start = time.time()
func = mod.get_function("add")
func(x_gpu, y_gpu, block=(1, 1, 1))
y_added = np.empty_like(y)
cuda.memcpy_dtoh(y_added, y_gpu)
print(f"---- {time.time() - start} seconds ----")

cuda.DeviceAllocation.free

---- 0.2184922695159912 seconds ----


<function DeviceAllocation.free>

In [23]:
start = time.time()
max_error = 0.0
for i in range(0, N):
    global max_error
    max_error = max(max_error, (y_added[i] - 3.0))
print(f"Max error: {max_error}")
print(f"---- {time.time() - start} seconds ----")

Max error: 4.674736414298997e+18
---- 2.9716413021087646 seconds ----


### ZOMG! It worked! 🙀🙀🙀
Alright, now let's use multiple threads!